<a href="https://colab.research.google.com/github/krishnaaxo/NLP/blob/main/NLP_Using_Deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/NLP'

In [9]:
%cd /content/drive/MyDrive/NLP

/content/drive/MyDrive/NLP


In [10]:
!kaggle competitions download -c jigsaw-multilingual-toxic-comment-classification

 72% 27.0M/37.3M [00:00<00:00, 63.8MB/s]
100% 37.3M/37.3M [00:00<00:00, 83.3MB/s]
100% 29.8M/29.8M [00:00<00:00, 60.7MB/s]

  0% 0.00/3.44M [00:00<?, ?B/s]
100% 3.44M/3.44M [00:00<00:00, 56.4MB/s]
 99% 79.0M/79.6M [00:01<00:00, 77.1MB/s]
100% 79.6M/79.6M [00:01<00:00, 73.7MB/s]
100% 649M/650M [00:08<00:00, 58.9MB/s]
100% 650M/650M [00:08<00:00, 84.3MB/s]
 89% 11.0M/12.4M [00:00<00:00, 41.7MB/s]
100% 12.4M/12.4M [00:00<00:00, 39.8MB/s]
  0% 0.00/1.35M [00:00<?, ?B/s]
100% 1.35M/1.35M [00:00<00:00, 43.8MB/s]
 99% 289M/292M [00:03<00:00, 92.4MB/s]
100% 292M/292M [00:03<00:00, 85.4MB/s]
  0% 0.00/612k [00:00<?, ?B/s]
100% 612k/612k [00:00<00:00, 19.7MB/s]


In [11]:
!unzip \*.zip && rm *.zip

Archive:  jigsaw-toxic-comment-train.csv.zip
  inflating: jigsaw-toxic-comment-train.csv  

Archive:  test-processed-seqlen128.csv.zip
  inflating: test-processed-seqlen128.csv  

Archive:  validation-processed-seqlen128.csv.zip
  inflating: validation-processed-seqlen128.csv  

Archive:  jigsaw-toxic-comment-train-processed-seqlen128.csv.zip
  inflating: jigsaw-toxic-comment-train-processed-seqlen128.csv  

Archive:  jigsaw-unintended-bias-train-processed-seqlen128.csv.zip
  inflating: jigsaw-unintended-bias-train-processed-seqlen128.csv  

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  validation.csv.zip
  inflating: validation.csv          

Archive:  jigsaw-unintended-bias-train.csv.zip
  inflating: jigsaw-unintended-bias-train.csv  

8 archives were successfully processed.


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [6]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.109.97.34:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.109.97.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.109.97.34:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [12]:
train = pd.read_csv('/content/drive/MyDrive/NLP/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/content/drive/MyDrive/NLP/validation.csv')
test = pd.read_csv('/content/drive/MyDrive/NLP/test.csv')

In [13]:
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [16]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [21]:
train=train.drop(['severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],axis=1)

In [22]:
train

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1


In [23]:
validation

,id,comment_text,lang,toxic
0,0,Este usuario ni siquiera llega al rango de ...,es,0
1,1,Il testo di questa voce pare esser scopiazzato...,it,0
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,es,1
3,3,Bu maddenin alt başlığı olarak uluslararası i...,tr,0
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,tr,0
...,...,...,...,...
7995,7995,Il fatto è che la pagina dei personaggi minor...,it,0
7996,7996,El imbesil ete dela luna no se entera ni ostia...,es,1
7997,7997,olum sız manyakmısınz siz adam sıze sanal yıld...,tr,1
7998,7998,El mapa del reinado de Alhaken esta ligerament...,es,0


In [24]:
validation=validation.drop(['lang'],axis=1)
validation

,id,comment_text,toxic
0,0,Este usuario ni siquiera llega al rango de ...,0
1,1,Il testo di questa voce pare esser scopiazzato...,0
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,1
3,3,Bu maddenin alt başlığı olarak uluslararası i...,0
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,0
...,...,...,...
7995,7995,Il fatto è che la pagina dei personaggi minor...,0
7996,7996,El imbesil ete dela luna no se entera ni ostia...,1
7997,7997,olum sız manyakmısınz siz adam sıze sanal yıld...,1
7998,7998,El mapa del reinado de Alhaken esta ligerament...,0


In [25]:
test

,id,content,lang
0,0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru
2,2,"Quindi tu sei uno di quelli conservativi , ...",it
3,3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
4,4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr
...,...,...,...
63807,63807,"No, non risponderò, come preannunciato. Prefer...",it
63808,63808,"Ciao, I tecnici della Wikimedia Foundation sta...",it
63809,63809,innnazitutto ti ringrazio per i ringraziamenti...,it
63810,63810,Kaç olumlu oy gerekiyor? Şu an 7 oldu. Hayır...,tr


In [26]:
train = train.loc[:12000,:]
train.shape

(12001, 3)

In [27]:
train

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
11996,1fc25487b9b9187f,February 2006 (UTC)\n\nThank you for fixing th...,0
11997,1fc3e5540e2605d1,Erased another silly message.,0
11998,1fc4c99c34cf0e07,"""\n\n Helly Jimmy, well; fuck you. \n\nAccordi...",1
11999,1fc8d9f16da8cccc,"Why don't you run cry to your mommy, little As...",1


# Count

In [28]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

In [29]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [30]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values, 
                                                  stratify=train.toxic.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [31]:
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

# Simple RNN

In [32]:
%%time
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1500, 300)         13049100  
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               40100     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 13,089,301
Trainable params: 13,089,301
Non-trainable params: 0
_________________________________________________________________
CPU times: user 284 ms, sys: 26.5 ms, total: 310 ms
Wall time: 875 ms


In [37]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64) 

Epoch 1/5
150/150 [==============================] - 12s 51ms/step - loss: 0.3074 - accuracy: 0.9021
Epoch 2/5
150/150 [==============================] - 8s 51ms/step - loss: 0.1390 - accuracy: 0.9488
Epoch 3/5
150/150 [==============================] - 8s 51ms/step - loss: 0.0209 - accuracy: 0.9948
Epoch 4/5
150/150 [==============================] - 7s 50ms/step - loss: 0.0031 - accuracy: 0.9997
Epoch 5/5
150/150 [==============================] - 8s 51ms/step - loss: 8.3280e-04 - accuracy: 1.0000


In [38]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

Auc: 0.83%


In [39]:
scores_model = []
scores_model.append({'Model': 'SimpleRNN','AUC_Score': roc_auc(scores,yvalid)})

# LSTM

In [42]:
%%time
with strategy.scope():
    
    # A simple LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))

    model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1500, 300)         13049100  
_________________________________________________________________
lstm (LSTM)                  (None, 100)               160400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 13,209,601
Trainable params: 13,209,601
Non-trainable params: 0
_________________________________________________________________
CPU times: user 374 ms, sys: 38.7 ms, total: 412 ms
Wall time: 1.08 s


In [44]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64)

Epoch 1/5
150/150 [==============================] - 25s 116ms/step - loss: 0.3670 - accuracy: 0.8968
Epoch 2/5
150/150 [==============================] - 18s 117ms/step - loss: 0.1139 - accuracy: 0.9598
Epoch 3/5
150/150 [==============================] - 17s 116ms/step - loss: 0.0529 - accuracy: 0.9817
Epoch 4/5
150/150 [==============================] - 17s 116ms/step - loss: 0.0151 - accuracy: 0.9953
Epoch 5/5
150/150 [==============================] - 17s 116ms/step - loss: 0.0063 - accuracy: 0.9987


In [45]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

Auc: 0.93%


In [46]:
scores_model.append({'Model': 'LSTM','AUC_Score': roc_auc(scores,yvalid)})

# GRU

In [48]:
%%time
with strategy.scope():
    # GRU with glove embeddings and two dense layers
     model = Sequential()
     model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
     model.add(SpatialDropout1D(0.3))
     model.add(GRU(300))
     model.add(Dense(1, activation='sigmoid'))

     model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])   
    
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1500, 300)         13049100  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1500, 300)         0         
_________________________________________________________________
gru_1 (GRU)                  (None, 300)               541800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 301       
Total params: 13,591,201
Trainable params: 13,591,201
Non-trainable params: 0
_________________________________________________________________
CPU times: user 274 ms, sys: 41 ms, total: 315 ms
Wall time: 457 ms


In [52]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64)

Epoch 1/5
150/150 [==============================] - 24s 125ms/step - loss: 0.3375 - accuracy: 0.8978
Epoch 2/5
150/150 [==============================] - 19s 124ms/step - loss: 0.1052 - accuracy: 0.9638
Epoch 3/5
150/150 [==============================] - 19s 124ms/step - loss: 0.0425 - accuracy: 0.9864
Epoch 4/5
150/150 [==============================] - 19s 124ms/step - loss: 0.0079 - accuracy: 0.9984
Epoch 5/5
150/150 [==============================] - 19s 125ms/step - loss: 0.0029 - accuracy: 0.9995


In [53]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

Auc: 0.92%


In [54]:
scores_model.append({'Model': 'GRU','AUC_Score': roc_auc(scores,yvalid)})

In [55]:
%%time
with strategy.scope():
    # A simple bidirectional LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1500, 300)         13049100  
_________________________________________________________________
bidirectional (Bidirectional (None, 600)               1442400   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 601       
Total params: 14,492,101
Trainable params: 1,443,001
Non-trainable params: 13,049,100
_________________________________________________________________
CPU times: user 768 ms, sys: 1.55 s, total: 2.31 s
Wall time: 6.95 s


In [56]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64)

Epoch 1/5
150/150 [==============================] - 75s 457ms/step - loss: 0.3632 - accuracy: 0.9040
Epoch 2/5
150/150 [==============================] - 68s 456ms/step - loss: 0.3181 - accuracy: 0.9038
Epoch 3/5
150/150 [==============================] - 68s 457ms/step - loss: 0.3159 - accuracy: 0.9055
Epoch 4/5
150/150 [==============================] - 68s 457ms/step - loss: 0.3283 - accuracy: 0.8991
Epoch 5/5
150/150 [==============================] - 68s 457ms/step - loss: 0.3222 - accuracy: 0.9019


In [57]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

Auc: 0.50%


In [58]:
scores_model.append({'Model': 'Bi-directional LSTM','AUC_Score': roc_auc(scores,yvalid)})

In [59]:
results = pd.DataFrame(scores_model).sort_values(by='AUC_Score',ascending=False)
results.style.background_gradient(cmap='Blues')

,Model,AUC_Score
1,LSTM,0.928264
2,GRU,0.923894
0,SimpleRNN,0.831942
3,Bi-directional LSTM,0.500000


In [60]:
fig = go.Figure(go.Funnelarea(
    text =results.Model,
    values = results.AUC_Score,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()